In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install pyldavis

In [10]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models import LdaModel, CoherenceModel
from gensim.models.wrappers import LdaMallet
import pprint

# spacy for lemmatization
import spacy
from spacy.lang.en import English

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
%%time 
reviews_datasets=pd.read_csv('D:\so-data\PostsChosenV1.csv' , index_col='Id', )
# postTagsRawData = pd.read_csv('PostTagsV3.csv', chunksize=1000)
# postTagsRawData = pd.read_excel('PostsTags2.xlsx')
print(reviews_datasets)

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<unknown>:1: DeprecationWarning: invalid escape sequence \s


          AcceptedAnswerId  AnswerCount  \
Id                                        
1468404            1549052            3   
1583856            1634592           14   
1914947            1915407            4   
1961341            1961838            1   
2039387            2044192            4   
...                    ...          ...   
66394506          66397036            1   
66394874          66394940            1   
66400722          66400807            1   
66402094          66403323            1   
66405059          66405291            2   

                                                       Body  \
Id                                                            
1468404   <p>Does mono touch allow you to do cross iPhon...   
1583856   <p>After sitting through a session today on Mo...   
1914947   <p>What is the best approach to implement tabs...   
1961341   <p>How can I check internet access with MonoTo...   
2039387   <p>I'm new to MonoTouch and iPhone development...   

<decorator-gen-55>:2: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.


In [12]:
reviews_datasets.dropna()
obody = reviews_datasets['Body']

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
reviews_datasets.head()
#print(obody)

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
Id,,,,,,,,,,,,,,,,,,,
1468404,1549052,3,<p>Does mono touch allow you to do cross iPhon...,NaN,0,NaN,2009-09-23 20:42:43.883,2,2009-10-10 20:52:40.330,2009-09-23 20:45:37.153,NaN,25981,139096,0,1,3,<iphone><communication><xamarin.ios>,Cross iPhone communication using mono touch,1464
1583856,1634592,14,<p>After sitting through a session today on Mo...,2019-09-23 16:34:03.127,1,NaN,2009-10-18 02:47:39.653,91,2019-09-23 14:03:02.387,2012-03-07 16:08:46.430,user1228,0,142017,0,1,273,<c#><objective-c><mono><xamarin.ios>,How to decide between MonoTouch and Objective-C?,28366
1914947,1915407,4,<p>What is the best approach to implement tabs...,NaN,0,NaN,2009-12-16 14:23:59.397,5,2019-05-21 10:06:27.997,2019-05-21 10:06:27.997,NaN,4751173,78101,0,1,6,<iphone><uiview><uiviewcontroller><xamarin.ios>,Web-like tabs for the iPhone,1053
1961341,1961838,1,<p>How can I check internet access with MonoTo...,NaN,1,NaN,2009-12-25 15:38:21.420,2,2012-05-23 07:10:33.650,2009-12-25 18:33:38.633,NaN,30618,34155,0,1,14,<iphone><xamarin.ios>,Check for internet access with monotouch,5548
2039387,2044192,4,<p>I'm new to MonoTouch and iPhone development...,NaN,1,NaN,2010-01-11 01:12:49.940,2,2014-01-19 15:43:12.283,2014-01-19 15:43:12.283,NaN,578411,247706,0,1,14,<iphone><xamarin.ios>,Images in Interface Builder when using MonoTouch,4257


In [14]:
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk

import re
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
import re

# Convert to list
data = reviews_datasets.Body.values.tolist()

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single and double quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('\", "', '', sent) for sent in data]
data = [re.sub('\\"', '', sent) for sent in data]
data = [re.sub('\"', '', sent) for sent in data]
data = [re.sub('[\\:"]', '', sent) for sent in data]

# Remove web links
data = [re.sub(r'^https?:\/\/.*[\r\n]*', '', sent) for sent in data]

print(data[:3])

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

In [8]:
# Tokenize words and text clean up
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:3])

C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['does', 'mono', 'touch', 'allow', 'you', 'to', 'do', 'cross', 'iphone', 'communication', 'iphones', 'in', 'the', 'same', 'room', 'if', 'so', 'whats', 'the', 'api'], ['after', 'sitting', 'through', 'session', 'today', 'on', 'mono', 'at', 'local', 'net', 'event', 'the', 'use', 'of', 'monotouch', 'was', 'touched', 'upon', 'as', 'an', 'alternative', 'for', 'iphone', 'development', 'being', 'very', 'comfortable', 'in', 'and', 'net', 'it', 'seems', 'like', 'an', 'appealing', 'option', 'despite', 'some', 'of', 'the', 'quirkiness', 'of', 'the', 'mono', 'stack', 'however', 'since', 'monotouch', 'costs', 'im', 'somewhat', 'torn', 'on', 'if', 'this', 'is', 'the', 'way', 'to', 'go', 'for', 'iphone', 'development', 'anyone', 'have', 'an', 'experience', 'developing', 'with', 'monotouch', 'and', 'objective', 'and', 'if', 'so', 'is', 'developing', 'with', 'monotouch', 'that', 'much', 'simpler', 'and', 'quicker', 'than', 'learning', 'objective', 'and', 'in', 'turn', 'worth', 'the'], ['what', 'is', 't

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=75)  


C:\tools\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-7a2a774a9fb7>", line 3, in <module>
    trigram = gensim.models.Phrases(bigram[data_words], threshold=75)
  File "C:\tools\Anaconda3\lib\site-packages\gensim\models\phrases.py", line 419, in __init__
    self.add_vocab(sentences)
  File "C:\tools\Anaconda3\lib\site-packages\gensim\models\phrases.py", line 565, in add_vocab
    min_reduce, vocab, total_words = self.learn_vocab(
  File "C:\tools\Anaconda3\lib\site-packages\gensim\models\phrases.py", line 500, in learn_vocab
    for sentence_no, sentence in enumerate(sentences):
  File "C:\tools\Anaconda3\lib\site-packages\gensim\interfaces.py", line 181, in __iter__
    yield self.obj[doc]
  File "C:\tools\Anaconda3\lib\site-packages\gensim\models\phrases.py", line 682, in __getitem__
    return _sentence2token(self, sentenc

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[3]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(data_lemmatized[:2])

reviews_datasets['topics']=data_lemmatized
reviews_datasets['obody']=obody
reviews_datasets.head(20)
#reset option to default value


reviews_datasets.to_csv("E:\\so-data\\output1.csv")

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]